# Lab 2 Part 3

In this lab, we'll continue working with [ECCO](https://www.ecco-group.org/products-ECCO-V4r4.htm), a state estimate, which is a type of model that combines observations and dynamical equations to estimate the state of the climate. ECCO is an ocean state estimate ocean between 1992 and 2018. Unlike the simple ocean model we developed at the end of Lab 1, ECCO accounts for horizontal variation, and it includes many more variables besides temperature. The major objective of this lab is to learn to use model output to answer questions about ocean and climate dynamics.

In this third part, we'll continue analyzing the meridional overturning circulation. **Quantitative** students will create an observing array for the AMOC and measure how its strength has changed over time.

**Both tracks are asked to save some plots. Create a separate document for these plots and give each plot a figure number and a descriptive caption. Refer to the figures by their figure number in the documents that you turn in, whether that is a Jupyter notebook (quantitative) or a written document (qualitative).**

Begin by running the code block below to import packages and set up plotting tools. If it runs correctly, you will see "Setup complete".

In [ ]:
%matplotlib ipympl
import math
import os
import requests
import datetime
import xgcm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from platform import system
from netrc import netrc
from urllib import request
from http.cookiejar import CookieJar
from io import StringIO
from warnings import filterwarnings
import ecco_v4_py as ecco
from ecco_v4_py import vector_calc, scalar_calc
from os.path import join,expanduser,exists,split

filterwarnings("ignore", category=FutureWarning)
downloads = '/oscar/data/eeps1400_24fall/DATA/ECCO_V4r4_PODAAC'

# Information to look up a variable in EarthData by name
all_variables = ['global_mean_barystatic_sea_level_anomaly', 'global_mean_sterodynamic_sea_level_anomaly', 'global_mean_sea_level_anomaly', 'Pa_global', 'xoamc', 'yoamc', 'zoamc', 'xoamp', 'yoamp', 'zoamp', 'mass', 'xcom', 'ycom', 'zcom', 'sboarea', 'xoamc_si', 'yoamc_si', 'zoamc_si', 'mass_si', 'xoamp_fw', 'yoamp_fw', 'zoamp_fw', 'mass_fw', 'xcom_fw', 'ycom_fw', 'zcom_fw', 'mass_gc', 'xoamp_dsl', 'yoamp_dsl', 'zoamp_dsl', 'CS', 'SN', 'rA', 'dxG', 'dyG', 'Depth', 'rAz', 'dxC', 'dyC', 'rAw', 'rAs', 'drC', 'drF', 'PHrefC', 'PHrefF', 'hFacC', 'hFacW', 'hFacS', 'maskC', 'maskW', 'maskS', 'DIFFKR', 'KAPGM', 'KAPREDI', 'SSH', 'SSHIBC', 'SSHNOIBC', 'ETAN', 'EXFatemp', 'EXFaqh', 'EXFuwind', 'EXFvwind', 'EXFwspee', 'EXFpress', 'EXFtaux', 'EXFtauy', 'oceTAUX', 'oceTAUY', 'EXFhl', 'EXFhs', 'EXFlwdn', 'EXFswdn', 'EXFqnet', 'oceQnet', 'SIatmQnt', 'TFLUX', 'EXFswnet', 'EXFlwnet', 'oceQsw', 'SIaaflux', 'EXFpreci', 'EXFevap', 'EXFroff', 'SIsnPrcp', 'EXFempmr', 'oceFWflx', 'SIatmFW', 'SFLUX', 'SIacSubl', 'SIrsSubl', 'SIfwThru', 'SIarea', 'SIheff', 'SIhsnow', 'sIceLoad', 'SIuice', 'SIvice', 'ADVxHEFF', 'ADVyHEFF', 'DFxEHEFF', 'DFyEHEFF', 'ADVxSNOW', 'ADVySNOW', 'DFxESNOW', 'DFyESNOW', 'oceSPflx', 'oceSPDep', 'MXLDEPTH', 'OBP', 'OBPGMAP', 'PHIBOT', 'UVEL', 'VVEL', 'WVEL', 'THETA', 'SALT', 'RHOAnoma', 'DRHODR', 'PHIHYD', 'PHIHYDcR', 'UVELMASS', 'VVELMASS', 'WVELMASS', 'Um_dPHdx', 'Vm_dPHdy', 'ADVx_TH', 'ADVy_TH', 'ADVr_TH', 'DFxE_TH', 'DFyE_TH', 'DFrE_TH', 'DFrI_TH', 'ADVx_SLT', 'ADVy_SLT', 'ADVr_SLT', 'DFxE_SLT', 'DFyE_SLT', 'DFrE_SLT', 'DFrI_SLT', 'oceSPtnd', 'UVELSTAR', 'VVELSTAR', 'WVELSTAR', 'GM_PsiX', 'GM_PsiY']
all_datasets = ['GMSL_TIME_SERIES', 'GMAP_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'GEOMETRY_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'SSH_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'STRESS_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'MIXED_LAYER_DEPTH_LLC0090GRID', 'OBP_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'BOLUS_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID']
datasets = pd.Series(['GMSL_TIME_SERIES', 'GMSL_TIME_SERIES', 'GMSL_TIME_SERIES', 'GMAP_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'MIXED_LAYER_DEPTH_LLC0090GRID', 'OBP_LLC0090GRID', 'OBP_LLC0090GRID', 'OBP_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'BOLUS_LLC0090GRID', 'BOLUS_LLC0090GRID', 'BOLUS_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID'],
                     index=all_variables)
timings = pd.Series(['Daily', 'Snapshot', 'Snapshot', 'None', 'None', 'All', 'Daily', 'Daily', 'Daily', 'Daily', 'All', 'All', 'Daily', 'Daily', 'Daily', 'All', 'Daily', 'All', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily'],
                    index=all_datasets)
granule_prefixes = pd.Series(['GLOBAL_MEAN_SEA_LEVEL', 'GLOBAL_MEAN_ATM_SURFACE_PRES', 'SBO_CORE_PRODUCTS', 'GRID_GEOMETRY', 'OCEAN_3D_MIXING_COEFFS', 'SEA_SURFACE_HEIGHT', 'ATM_SURFACE_TEMP_HUM_WIND_PRES', 'OCEAN_AND_ICE_SURFACE_STRESS', 'OCEAN_AND_ICE_SURFACE_HEAT_FLUX', 'OCEAN_AND_ICE_SURFACE_FW_FLUX', 'SEA_ICE_CONC_THICKNESS', 'SEA_ICE_VELOCITY', 'SEA_ICE_HORIZ_VOLUME_FLUX', 'SEA_ICE_SALT_PLUME_FLUX', 'OCEAN_MIXED_LAYER_DEPTH', 'OCEAN_BOTTOM_PRESSURE', 'OCEAN_VELOCITY', 'OCEAN_TEMPERATURE_SALINITY', 'OCEAN_DENS_STRAT_PRESS', 'OCEAN_3D_VOLUME_FLUX', 'OCEAN_3D_MOMENTUM_TEND', 'OCEAN_3D_TEMPERATURE_FLUX', 'OCEAN_3D_SALINITY_FLUX', 'OCEAN_BOLUS_VELOCITY', 'OCEAN_BOLUS_STREAMFUNCTION'],
                             index=all_datasets)

# Information to generate an xgcm grid
tile_connections = {'tile': {
    0: {'X': ((12, 'Y', False), (3, 'X', False)), 'Y': (None, (1, 'Y', False))},
    1: {'X': ((11, 'Y', False), (4, 'X', False)), 'Y': ((0, 'Y', False), (2, 'Y', False))},
    2: {'X': ((10, 'Y', False), (5, 'X', False)), 'Y': ((1, 'Y', False), (6, 'X', False))},
    3: {'X': ((0, 'X', False), (9, 'Y', False)), 'Y': (None, (4, 'Y', False))},
    4: {'X': ((1, 'X', False), (8, 'Y', False)), 'Y': ((3, 'Y', False), (5, 'Y', False))},
    5: {'X': ((2, 'X', False), (7, 'Y', False)), 'Y': ((4, 'Y', False), (6, 'Y', False))},
    6: {'X': ((2, 'Y', False), (7, 'X', False)), 'Y': ((5, 'Y', False), (10, 'X', False))},
    7: {'X': ((6, 'X', False), (8, 'X', False)), 'Y': ((5, 'X', False), (10, 'Y', False))},
    8: {'X': ((7, 'X', False), (9, 'X', False)), 'Y': ((4, 'X', False), (11, 'Y', False))},
    9: {'X': ((8, 'X', False), None), 'Y': ((3, 'X', False), (12, 'Y', False))},
    10: {'X': ((6, 'Y', False), (11, 'X', False)), 'Y': ((7, 'Y', False), (2, 'X', False))},
    11: {'X': ((10, 'X', False), (12, 'X', False)), 'Y': ((8, 'Y', False), (1, 'X', False))},
    12: {'X': ((11, 'X', False), None), 'Y': ((9, 'Y', False), (0, 'X', False))}
}}

# So that you don't have to remember whether to put dimension names in quotes or not
i, i_g, j, j_g, k, k_u, k_l, k_p1, tile, XC, YC, XG, YG, Z, Zp1, Zu, Zl, XC_bnds, YC_bnds, Z_bnds, tile, time, time_l = 'i', 'i_g', 'j', 'j_g', 'k', 'k_u', 'k_l', 'k_p1', 'tile', 'XC', 'YC', 'XG', 'YG', 'Z', 'Zp1', 'Zu', 'Zl', 'XC_bnds', 'YC_bnds', 'Z_bnds', 'tile', 'time', 'time_l'

# Used to select i, j, i_g, and j_g for quiver plots to space out data
skip = range(2, 88, 5)

# subplots[i] is the index of tile #i in the array of subplots
subplots = {
    'pacific': [(3, 0), (2, 0), (1, 0), (3, 1), (2, 1), (1, 1), (0, 2),
              (1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)],
    'atlantic': [(3, 2), (2, 2), (1, 2), (3, 3), (2, 3), (1, 3), (0, 2),
               (1, 0), (2, 0), (3, 0), (1, 1), (2, 1), (3, 1)],
}
# rotations[i] is the orientation of tile #i, as a multiple of 90 degrees
rotations = {
    'pacific': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'atlantic': [0, 0, 0, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1],
}

# Trigonometry for multiples of 90 degrees 
def cos90(angle):
    if angle % 4 == 0: return 1
    elif angle % 4 == 2: return -1
    else: return 0
def sin90(angle):
    if angle % 4 == 1: return 1
    elif angle % 4 == 3: return -1
    else: return 0

def adjust_timing(variable: str, timing: str) -> str:
    dataset = datasets[variable]
    if timing not in {'None', 'Monthly', 'Daily', 'Monthly Snapshot', 'Daily Snapshot'}:
        raise ValueError(str(timing) + ' is not a valid timing (select either Monthly, Daily, Monthly Snapshot, or Daily Snapshot)')
    elif timing in {'Monthly Snapshot', 'Daily Snapshot'} and timings[dataset] == 'Daily':
        raise ValueError('No snapshots available for ' + str(variable))
    elif timing in {'Monthly', 'Daily'} and timings[dataset] == 'Snapshot':
        raise ValueError('No monthly or daily averages available for ' + str(variable))
    elif timings[dataset] == 'None':
        return 'None'
    elif timing == 'None' and timings[dataset] == 'Snapshot':
        return 'Monthly Snapshot'
    elif timing == 'None' and timings[dataset] in {'Daily', 'All'}:
        return 'Monthly'
    else:
        return timing

def get_granule(granule: str, directory: str) -> str:
    file = os.path.join(directory, os.path.basename(granule))
    if not os.path.isfile(file):
        print('File not downloaded: ' + granule)
    return file

def ecco_dataset(dataset: str, start: datetime.date = None, end: datetime.date = None, timing: str = 'None'):
    short_timing_names = {'None': '', 'Monthly': '_MONTHLY', 'Daily': '_DAILY', 'Monthly Snapshot': '_SNAPSHOT', 'Daily Snapshot': '_SNAPSHOT'}
    long_timing_names = {'None': '', 'Monthly': '_mon_mean', 'Daily': '_day_mean', 'Monthly Snapshot': '_snap', 'Daily Snapshot': '_snap'}
    if timing not in short_timing_names:
        raise ValueError('Unrecognized timing: ' + str(timing))
    shortname = 'ECCO_L4_' + dataset + short_timing_names[timing] + '_V4R4'
    if 'LLC0090' in dataset:
        if timing == 'Monthly':
            start = datetime.date(start.year, start.month, 1)
            dates = [date.strftime('_%Y-%m') for date in pd.date_range(start, end, freq='MS')]
        elif timing == 'Daily':
            dates = [date.strftime('_%Y-%m-%d') for date in pd.date_range(start, end)]
        elif timing in {'Monthly Snapshot', 'Daily Snapshot'}:
            dates = [date.strftime('_%Y-%m-%dT000000') for date in pd.date_range(start, end)]
        elif timing == 'None':
            dates = ['']
        longnames = [granule_prefixes[dataset] + long_timing_names[timing] + date + '_ECCO_V4r4_native_llc0090.nc'
                    for date in dates]
    else:
        longnames = [granule_prefixes[dataset] + long_timing_names[timing] + '_ECCO_V4r4_1D.nc']
    granules = ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/' + shortname + '/' + longname
                for longname in longnames]
    granule_dir = downloads + '/' + shortname
    try: os.mkdir(granule_dir)
    except FileExistsError: pass
    files = [get_granule(granule, granule_dir) for granule in granules]
    array = xr.open_mfdataset(files, data_vars='minimal', coords='minimal', compat='override')
    if timing == 'Monthly':
        times = pd.DatetimeIndex(array.time)
        array = array.assign_coords(time=[str(t)[:7] for t in times])
    elif timing in {'Daily', 'Daily Snapshot', 'Monthly Snapshot'}:
        times = pd.DatetimeIndex(array.time)
        array = array.assign_coords(time=[str(t)[:10] for t in times])
    if timing == 'Monthly Snapshot':
        array = array.sel(time=[t for t in array.time.values if t[8:10] == '01'])
        array = array.assign_coords(time=[t[:7] for t in array.time.values])
    if timing in {'Daily Snapshot', 'Monthly Snapshot'}:
        array = array.rename(time=time_l)
    return array

def ecco_variable(variable: str, start: datetime.date = None, end: datetime.date = None, timing: str = 'None'):
    if variable not in all_variables:
        raise ValueError(str(variable) + ' is not an ECCO variable')
    timing = adjust_timing(variable, timing)
    if timing != 'None' and start is None and 'LLC0090' in datasets[variable]:
        raise ValueError('Enter a date to retrieve \'' + str(variable) + '\'')
    if type(start) == str:
        if len(start) == 7:
            start += '-01'
        start = datetime.datetime.strptime(start, '%Y-%m-%d')
    if type(end) == str:
        if len(end) == 7:
            end += '-01'
        end = datetime.datetime.strptime(end, '%Y-%m-%d')
    if end is None:
        end = start
    return ecco_dataset(datasets[variable], start, end, timing)[variable]

def print_value(array):
    if len(array.dims) > 1:
        dims = ', '.join(array.dims)
        raise ValueError('To get a single value, select or average along the remaining dimensions: ' + dims)
    else:
        value = array.values.item()
        if math.isnan(value):
            print('No value found (location is outside the bounds of the ocean)')
        else:
            if 'long_name' in array.attrs:
                print(array.long_name[:-1] + ': ' + str(value))
            else:
                print(value)
        for coord in {'XC', 'XG'}:
            if coord in array.coords:
                longitude = array[coord].values.item()
                print('Longitude: ' + str(abs(round(longitude, 3))) + ('°W' if longitude < 0 else '°E'))
                break
        for coord in {'YC', 'YG'}:
            if coord in array.coords:
                latitude = array[coord].values.item()
                print('Latitude: ' + str(abs(round(latitude, 3))) + ('°S' if latitude < 0 else '°N'))
                break
        for coord in {'Z', 'Zl', 'Zu', 'Zp1'}:
            if coord in array.coords:
                depth = array[coord].values.item()
                print('Depth: ' + str(round(-depth, 3)) + ' meters')
                break

def bounds(bottom, top): return range(bottom, top + 1)

geometry = ecco_dataset('GEOMETRY_LLC0090GRID')
xgcm_grid = xgcm.Grid(geometry, periodic=False, face_connections=tile_connections)

def interpolate(array, dim):
    if dim not in array.dims:
        raise ValueError(str(dim) + ' is not a dimension of the given variable')
    if len(array[dim]) < 2:
        raise ValueError('You need at least two coordinates to interpolate along a dimension')
    if dim == 'time':
        times = array[dim].values
        array = array.assign_coords({dim: range(len(times))})
        array = array.interp({dim: np.linspace(0.5, len(times) - 1.5, len(times) - 1)})
        return array.assign_coords({dim: times[1:]}).rename({dim: 'time_l'})
    if dim == 'time_l':
        times = array[dim].values
        array = array.assign_coords({dim: range(len(times))})
        array = array.interp({dim: np.linspace(0.5, len(times) - 1.5, len(times) - 1)})
        return array.assign_coords({dim: times[:-1]}).rename({dim: 'time'})
    grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(array.dims)
    if len(grid_dims) < 3 or any(len(array[dim]) < len(geometry[dim]) for dim in grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if dim in {'i', 'i_g', 'XC', 'XG'}:
        array_interp = xgcm_grid.interp(array.load(), 'X', keep_coords=True)
    elif dim in {'j', 'j_g', 'YC', 'YG'}:
        array_interp = xgcm_grid.interp(array.load(), 'Y', keep_coords=True)
    elif dim in {'k', 'k_u', 'k_l', 'k_p1', 'Z', 'Zp1', 'Zu', 'Zl'}:
        array_interp = xgcm_grid.interp(array.load(), 'Z', boundary='fill', fill_value=0, keep_coords=True)
    else: raise ValueError('Cannot interpolate along ' + str(dim))
    if 'time' in array.coords: array_interp = array_interp.assign_coords(time=array.time)
    elif 'time_l' in array.coords: array_interp = array_interp.assign_coords(time_l=array.time_l)
    return array_interp

def interpolate_2d(u, v):
    if {'i', 'j_g'} & set(u.dims):
        raise ValueError('The first input to interpolate_2d must be on the u-grid')
    if {'i_g', 'j'} & set(v.dims):
        raise ValueError('The second input to interpolate_2d must be on the v-grid')
    u_grid_dims, v_grid_dims = {'i_g', 'j', 'tile'}, {'i', 'j_g', 'tile'}
    if not (u_grid_dims <= set(u.dims)) or any(len(u[dim]) < len(geometry[dim]) for dim in u_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if not (v_grid_dims <= set(v.dims)) or any(len(v[dim]) < len(geometry[dim]) for dim in v_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    uv_interp = xgcm_grid.interp_2d_vector({'X': u.load(), 'Y': v.load()}, boundary='extend')
    u_interp, v_interp = uv_interp['X'], uv_interp['Y']
    if 'time' in u.coords: u_interp = u_interp.assign_coords(time=u.time)
    elif 'time_l' in u.coords: u_interp = u_interp.assign_coords(time_l=u.time_l)
    if 'time' in v.coords: v_interp = v_interp.assign_coords(time=v.time)
    elif 'time_l' in v.coords: v_interp = v_interp.assign_coords(time_l=v.time_l)
    return u_interp, v_interp

def difference(array, dim):
    if dim not in array.dims:
        raise ValueError(str(dim) + ' is not a dimension of the given variable')
    if len(array[dim]) < 2:
        raise ValueError('You need at least two coordinates to calculate difference along a dimension')
    if dim == 'time':
        return array.diff('time', label='upper').rename({'time': 'time_l'})
    if dim == 'time_l':
        return array.diff('time_l', label='lower').rename({'time_l': 'time'})
    grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(array.dims)
    if len(grid_dims) < 3 or any(len(array[dim]) < len(geometry[dim]) for dim in grid_dims):
        raise ValueError('You must calculate difference before selecting along grid dimensions')
    if dim in {'i', 'i_g', 'XC', 'XG'}:
        array_diff = xgcm_grid.diff(array.load(), 'X', keep_coords=True)
    elif dim in {'j', 'j_g', 'YC', 'YG'}:
        array_diff = xgcm_grid.diff(array.load(), 'Y', keep_coords=True)
    elif dim in {'k', 'k_u', 'k_l', 'k_p1', 'Z', 'Zp1', 'Zu', 'Zl'}:
        array_diff = -xgcm_grid.diff(array.load(), 'Z', boundary='fill', fill_value=0, keep_coords=True)
    else: raise ValueError('Cannot calculate difference along ' + str(dim))
    if 'time' in array.coords: array_diff = array_diff.assign_coords(time=array.time)
    elif 'time_l' in array.coords: array_diff = array_diff.assign_coords(time_l=array.time_l)
    return array_diff

def difference_2d(u, v):
    if {'i', 'j_g'} & set(u.dims):
        raise ValueError('The first input to interpolate_2d must be on the u-grid')
    if {'i_g', 'j'} & set(v.dims):
        raise ValueError('The second input to interpolate_2d must be on the v-grid')
    u_grid_dims, v_grid_dims = {'i_g', 'j', 'tile'}, {'i', 'j_g', 'tile'}
    if not (u_grid_dims <= set(u.dims)) or any(len(u[dim]) < len(geometry[dim]) for dim in u_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if not (v_grid_dims <= set(v.dims)) or any(len(v[dim]) < len(geometry[dim]) for dim in v_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    uv_diff = xgcm_grid.diff_2d_vector({'X': u.load(), 'Y': v.load()}, boundary='extend')
    u_diff, v_diff = uv_diff['X'], uv_diff['Y']
    if 'time' in u.coords: u_diff = u_diff.assign_coords(time=u.time)
    elif 'time_l' in u.coords: u_diff = u_diff.assign_coords(time_l=u.time_l)
    if 'time' in v.coords: v_diff = v_diff.assign_coords(time=v.time)
    elif 'time_l' in v.coords: v_diff = v_diff.assign_coords(time_l=v.time_l)
    return u_diff, v_diff

def colormap(data: xr.DataArray):
    cmin = np.nanpercentile(data, 10)
    cmax = np.nanpercentile(data, 90)
    if cmin < 0 and cmax > 0:
        cmax = np.nanpercentile(np.abs(data), 90)
        cmin = -cmax
        cmap = 'RdBu_r'
    else:
        cmap = 'viridis'

    return cmap, cmin, cmax

dimension_descriptions = {'i': 'Tile x-coordinate', 'j': 'Tile y-coordinate', 'k': 'Tile z-coordinate', 'Z': 'Depth (m)', 'tile': 'Plot area', 'time': 'Date'}
land_mask = mpl.colors.LinearSegmentedColormap.from_list('land_mask', ['#e0f0a0', '#ffffff'])

def update_plot(fig, data, x, y, selection, ocean_focus=None):
    names = data.data_vars.keys()
    title = widgets.Text(description='Plot title:')
    adjust_widgets = [title]
    ckind = data.c.dtype.kind
    if 'long_name' in data.c.attrs and 'vertical open fraction' in data.c.attrs['long_name']:
        ckind = 'b'
    cmap = widgets.Dropdown(description='Color map:', options=[
        ('viridis', 'viridis'), ('inferno', 'inferno'), ('cividis', 'cividis'), ('gray', 'binary'), ('gray (inverted)', 'gray'),
        ('pale', 'pink'), ('heat', 'gist_heat'), ('red-blue', 'RdBu_r'), ('seismic', 'seismic'), ('spectral', 'Spectral'),
        ('land mask', land_mask)
    ])
    if ckind == 'f':
        clabel = widgets.Text(description='Color units:')
        adjust_widgets.append(clabel)
    if {'u', 'v'} <= data.data_vars.keys():
        uvlabel = widgets.Text(description='Arrow units:')
        adjust_widgets.append(uvlabel)
    if ckind == 'f':
        adjust_widgets.append(cmap)
        if {'u', 'v'} <= data.data_vars.keys():
            acolor = widgets.Dropdown(description='Arrow color:', options=[('Black', 'k'), ('White', 'w')], value='k')
            adjust_widgets.append(acolor)
    display(widgets.HBox(adjust_widgets))

    fig.clf()
    # Select time/depth if possible before interpolating
    for dim in {'time', 'k'}:
        if dim in selection and dim in data.dims:
            data = data.sel({dim: selection[dim]})
    variables = dict(data.astype(float).data_vars)
    for (name, var) in variables.items():
        for dim in {'i_g', 'j_g', 'k_u', 'k_l', 'k_p1', 'time_l'}:
            if dim in var.dims:
                variables[name] = interpolate(var, dim)
    data = xr.Dataset(variables)
    # Second pass selection after interpolation changes dimensions
    for (dim, val) in selection.items():
        if dim in data.dims:
            data = data.sel({dim: val})
    if 'Z' in (x, y): data['Z'] = -data['Z']
    if ckind == 'f':
        cmap.value, cmin, cmax = colormap(data['c'])
    elif ckind == 'b':
        cmap.value, cmin, cmax = land_mask, 0, 1
    if {'u', 'v'} <= set(data.data_vars):
        x_skip, y_skip = math.ceil(len(data[x]) / 20), math.ceil(len(data[y]) / 20)
        quiver_x, quiver_y = data[x][(x_skip//2)::x_skip], data[y][(y_skip//2)::y_skip]
        uvmax = max(np.nanpercentile(np.abs(data.u), 90), np.nanpercentile(np.abs(data.v), 90))
    if 'tile' in data.dims:
        axes = fig.subplots(4, 4)
        if ckind == 'f':
            fig.set_size_inches(12.5, 10.1)
        elif ckind == 'b':
            fig.set_size_inches(10, 10.1)
        fig.subplots_adjust(wspace=0, hspace=0)
        for ax in axes.ravel():
            ax.axis('off')
        axes = [axes[row][col] for (row, col) in subplots[ocean_focus]]
        title.observe(lambda change: fig.suptitle(change['new'], x=0.435, y=0.92), names='value')
        meshes, quivers = [], []
        for tile, ax in enumerate(axes):
            if tile not in data.tile: continue
            ax.axis('on')
            ax.set_aspect('equal')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            c_rotated = np.rot90(data.c.sel(tile=tile).load(), rotations[ocean_focus][tile])
            meshes.append(ax.pcolormesh(data[x], data[y], c_rotated, cmap=cmap.value, vmin=cmin, vmax=cmax))
            if {'u', 'v'} <= set(data.data_vars):
                # Rotate head of each vector around the tile to the correct orientation
                u_rotated = np.rot90(data.u.sel({'tile': tile, x: quiver_x, y: quiver_y}), rotations[ocean_focus][tile])
                v_rotated = np.rot90(data.v.sel({'tile': tile, x: quiver_x, y: quiver_y}), rotations[ocean_focus][tile])
                # Rotate tail of each vector around the head by the same amount
                u_adjusted = u_rotated * cos90(rotations[ocean_focus][tile]) + v_rotated * sin90(rotations[ocean_focus][tile])
                v_adjusted = v_rotated * cos90(rotations[ocean_focus][tile]) - u_rotated * sin90(rotations[ocean_focus][tile])
                quivers.append(ax.quiver(quiver_x, quiver_y, u_adjusted, v_adjusted, scale=20*uvmax, width=0.006, clip_on=False))
        if ckind == 'f':
            cbar = fig.colorbar(meshes[0], ax=axes)
            clabel.observe(lambda change: cbar.set_label(change['new']), names='value')
            cmap.observe(lambda change: [mesh.set_cmap(change['new']) for mesh in meshes], names='value')
            if {'u', 'v'} <= set(data.data_vars):
                [quiver.set_color(acolor.value) for quiver in quivers]
                acolor.observe(lambda change: [quiver.set_color(change['new']) for quiver in quivers], names='value')
        if {'u', 'v'} <= set(data.data_vars):
            quiverkey = axes[6].quiverkey(quivers[6], 1.5, 0.5, 5*uvmax, f'{5*uvmax:.5g}')
            def set_quiverkey_label(change):
                nonlocal quiverkey
                quiverkey.remove()
                label = f'{5*uvmax:.5g}'
                if len(change['new']) > 0:
                    label += ' ' + change['new']
                quiverkey = axes[6].quiverkey(quivers[6], 1.5, 0.5, 5*uvmax, label)
            uvlabel.observe(set_quiverkey_label, names='value')
    else:
        ax = fig.subplots()
        if ckind == 'f':
            fig.set_size_inches(6.5, 5)
        elif ckind == 'b':
            fig.set_size_inches(5, 5)
        ax.set_xlabel(dimension_descriptions[x])
        ax.set_ylabel(dimension_descriptions[y])
        title.observe(lambda change: ax.set_title(change['new']), names='value')
        transpose = (x != data.c.dims[1] and y != data.c.dims[0])
        if (y in {'k', 'Z'}) or (transpose and y == 'i'):
            ax.yaxis.set_inverted(True)
            if 'v' in data.data_vars:
                data['v'] = -data['v']
        mesh_c = data.c.values
        if transpose: mesh_c = mesh_c.T
        mesh = ax.pcolormesh(data[x], data[y], mesh_c, cmap=cmap.value, vmin=cmin, vmax=cmax)
        if ckind == 'f':
            cbar = fig.colorbar(mesh)
            clabel.observe(lambda change: cbar.set_label(change['new']), names='value')
            cmap.observe(lambda change: mesh.set_cmap(change['new']), names='value')
        if {'u', 'v'} <= names:
            quiver_u = data.u.where(data[x].isin(quiver_x), drop=True).where(data[y].isin(quiver_y), drop=True)
            quiver_v = data.v.where(data[x].isin(quiver_x), drop=True).where(data[y].isin(quiver_y), drop=True)
            quiver_u, quiver_v = quiver_u.values, quiver_v.values
            if transpose: quiver_u, quiver_v = quiver_u.T, quiver_v.T
            quiver = ax.quiver(quiver_x, quiver_y, quiver_u, quiver_v, scale=20*uvmax, width=0.006)
            quiverkey = ax.quiverkey(quiver, 0.95, 1.05, 2*uvmax, f'{2*uvmax:.5g} ')
            def set_quiverkey_label(change):
                nonlocal quiverkey
                quiverkey.remove()
                label = f'{2*uvmax:.5g}'
                if len(change['new']) > 0:
                    label += ' ' + change['new']
                quiverkey = ax.quiverkey(quiver, 0.95, 1.05, 2*uvmax, label)
            uvlabel.observe(set_quiverkey_label, names='value')
            if ckind == 'f':
                quiver.set_color(acolor.value)
                acolor.observe(lambda change: quiver.set_color(change['new']), names='value')
        if x in {'time', 'time_l'}:
            ax.set_xticks(ax.get_xticks()[::3])

def make_coords_widget(selection, coords):
    output = widgets.Output()
    def show_coords(change):
        if change['new'] == 'Choose a value:':
            with output: display(coords)
        else:
            output.clear_output()
    selection.observe(show_coords, names='value')
    return output, show_coords

def plot(c: xr.DataArray = None, u: xr.DataArray = None, v: xr.DataArray = None):
    # If there is no color plot, plot land vs. ocean instead
    if c is None:
        c = ecco_variable('hFacC')
        if (u is None or 'k' not in u.dims) and (v is None or 'k' not in v.dims):
            c = c.sel(k=0)
    # If one of the arrow components isn't used, make it zero
    if u is not None and v is None:
        v = xr.DataArray(0, coords=u.coords, dims=u.dims)
    if v is not None and u is None:
        u = xr.DataArray(0, coords=v.coords, dims=v.dims)
        print(u)
    # plt.close() # Close other open plots to avoid having too many plots open at once
    # Merge variables into one dataset in order to perform uniform selection
    data = xr.Dataset({x_name: x for (x_name, x) in {'c': c, 'u': u, 'v': v}.items() if x is not None})
    if len(set(data.dims) - {'tile'}) < 2:
        raise ValueError('Must have at least two dimensions to make a plot')
    if any(len(data[dim]) == 0 for dim in data.dims):
        raise ValueError('Dimension with zero length')
    if {'i_g', 'j_g', 'k_l', 'k_u', 'k_p1', 'time_l'} & set(data.dims):
        grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(data.dims)
        if len(grid_dims) < 3 or any(len(data[dim]) < len(geometry[dim]) for dim in grid_dims):
            raise ValueError('In order for plotting to work correctly, you have to interpolate to grid cell centers before selecting along grid dimensions')
    selection_widgets = dict()
    selection_hboxes = []
    if 'tile' in data.dims:
        tile_options = [('Tile ' + str(tile), tile) for tile in data.tile.values]
        # Multi-tile plots only make sense if the data variables have both x- and y-coordinates
        if {'i', 'i_g'} & set(data.dims) and {'j', 'j_g'} & set(data.dims):
            tile_options = [('All tiles (Atlantic)', -1), ('All tiles (Pacific)', -2)] + tile_options
        tile_selection = widgets.Dropdown(description = 'Plot area:', options = tile_options)
        all_tiles_widgets = dict()
    if {'i', 'i_g'} & set(data.dims):
        i_selection = widgets.Dropdown(
            description = 'Tile x-coord:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Plot on x-axis',
        )
        i_coords = widgets.IntSlider(min=0, max=89)
        i_output, i_show_coords = make_coords_widget(i_selection, i_coords)
        selection_widgets['i'] = [i_selection, i_coords, i_output, i_show_coords]
        selection_hboxes.append(widgets.HBox([i_selection, i_output]))
    if {'j', 'j_g'} & set(data.dims):
        j_selection = widgets.Dropdown(
            description = 'Tile y-coord:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Plot on y-axis',
        )
        j_coords = widgets.IntSlider(min=0, max=89)
        j_output, j_show_coords = make_coords_widget(j_selection, j_coords)
        selection_widgets['j'] = [j_selection, j_coords, j_output, j_show_coords]
        selection_hboxes.append(widgets.HBox([j_selection, j_output]))
    if {'k', 'k_l', 'k_u', 'k_p1'} & set(data.dims):
        k_selection = widgets.Dropdown(
            description = 'Depth:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Choose a value:',
        )
        k_coords = widgets.SelectionSlider(options=[(str(int(-k)) + ' m', i) for (i, k) in enumerate(geometry.Z.values)])
        k_proportional = widgets.Checkbox(description='Proportional axis', value=False)
        k_output = widgets.Output()
        def k_show_coords(change):
            if change['new'] == 'Choose a value:':
                k_output.clear_output()
                with k_output: display(k_coords)
            elif change['old'] == 'Choose a value:':
                k_output.clear_output()
                with k_output: display(k_proportional)
        k_selection.observe(k_show_coords, names='value')
        selection_widgets['k'] = [k_selection, k_coords, k_output, k_show_coords]
        selection_hboxes.append(widgets.HBox([k_selection, k_output]))
        if 'tile' in data.dims:
            all_tiles_widgets['k'] = widgets.SelectionSlider(description='Depth:', options=k_coords.options)
    for dim in {'time', 'time_l'}:
        if dim in data.dims:
            t_selection = widgets.Dropdown(
                description = 'Date:',
                options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
                value = 'Choose a value:',
            )
            t_coords = widgets.SelectionSlider(options=data[dim].values)
            t_output, t_show_coords = make_coords_widget(t_selection, t_coords)
            selection_widgets['time'] = [t_selection, t_coords, t_output, t_show_coords]
            selection_hboxes.append(widgets.HBox([t_selection, t_output]))
            if 'tile' in data.dims:
                all_tiles_widgets['time'] = widgets.SelectionSlider(description='Date:', options=t_coords.options)
            break

    selection_output = widgets.Output()
    # 'change' means a change to the tile_selection widget's value (since tile_selection observes this function)
    def set_selection_widgets(change):
        selection_output.clear_output()
        if change['new'] < 0:
            with selection_output: display(*all_tiles_widgets.values())
        else:
            with selection_output: display(*selection_hboxes)
            for [selection, _, _, show_coords] in selection_widgets.values():
                # make coordinate sliders appear initially
                show_coords({'new': selection.value, 'old': 'Choose a value:'})
    set_selection_widgets({'new': tile_selection.value if 'tile' in data.dims else 0})
    if 'tile' in data.dims:
        tile_selection.observe(set_selection_widgets, names='value')

    plot_button = widgets.Button(description='Plot')
    clear_button = widgets.Button(description='Clear plot')
    plot_status = widgets.Label(value='')
    output = widgets.Output()
    fig = plt.figure()
    fig.set_size_inches(0.01, 0.01)

    def on_plot_button(_):
        plot_status.value = ''
        if 'tile' not in data.dims or tile_selection.value >= 0:
            selection = {dim: coords_widget.value
                         for (dim, [selection_widget, coords_widget, _, _]) in selection_widgets.items()
                         if selection_widget.value == 'Choose a value:'}
            if 'tile' in data.dims:
                selection['tile'] = tile_selection.value

            xaxis = [dim for (dim, [selection_widget, _, _, _]) in selection_widgets.items()
                     if selection_widget.value == 'Plot on x-axis']
            if len(xaxis) != 1:
                plot_status.value = 'One dimension must be selected to plot on the x-axis'
                return
            else: xaxis = xaxis[0]
            if xaxis == 'k' and k_proportional.value: xaxis = 'Z'

            yaxis = [dim for (dim, [selection_widget, _, _, _]) in selection_widgets.items()
                     if selection_widget.value == 'Plot on y-axis']
            if len(yaxis) != 1:
                plot_status.value = 'One dimension must be selected to plot on the y-axis'
                return
            else: yaxis = yaxis[0]
            if yaxis == 'k' and k_proportional.value: yaxis = 'Z'
        else:
            selection = {dim: widget.value for (dim, widget) in all_tiles_widgets.items()}
            xaxis, yaxis = 'i', 'j'
        output.clear_output()
        with output:
            if 'tile' not in data.dims or tile_selection.value >= 0: update_plot(fig, data, xaxis, yaxis, selection, None)
            elif tile_selection.value == -1: update_plot(fig, data, xaxis, yaxis, selection, 'atlantic')
            elif tile_selection.value == -2: update_plot(fig, data, xaxis, yaxis, selection, 'pacific')

    def on_clear_button(_):
        output.clear_output()
        fig.clf()
        fig.set_size_inches(0.01, 0.01)

    plot_button.on_click(on_plot_button)
    clear_button.on_click(on_clear_button)
    if 'tile' in data.dims:
        display(tile_selection)
    display(selection_output, widgets.HBox([plot_button, clear_button, plot_status]), output)
    plt.show()

def plot_utility():
    # plt.close()
    color = widgets.Text(description='Color plot:', value='THETA')
    quiver_x = widgets.Text(description='Arrow plot x:', value='UVELMASS')
    quiver_y = widgets.Text(description='Arrow plot y:', value='VVELMASS')
    hbox1 = widgets.HBox([color, quiver_x, quiver_y])
    start = widgets.DatePicker(description='Start date:', value=datetime.date(2017, 1, 1))
    end = widgets.DatePicker(description='End date:', value=datetime.date(2017, 1, 10))
    timing = widgets.Dropdown(options=['Monthly', 'Daily', 'Snapshot'], value='Daily', description='Timing:')
    hbox2 = widgets.HBox([start, end, timing])
    load_button = widgets.Button(description='Load data')
    clear_button = widgets.Button(description='Clear data')
    load_status = widgets.Label(value='')
    hbox3 = widgets.HBox([load_button, clear_button, load_status])
    output = widgets.Output()
    
    def on_load_button(_):
        if not (color.value or quiver_x.value or quiver_y.value):
            load_status.value = 'Enter variable names above'
        elif not (start.value and end.value):
            load_status.value = 'Enter start and end dates'
        elif start.value > end.value:
            load_status.value = 'Start date must be before end date'
        elif start.value < np.datetime64('1992-01-01'):
            load_status.value = 'Start date must not be before 1992'
        elif end.value >= np.datetime64('2018-01-01'):
            load_status.value = 'End date must not be after 2017'
        else:
            load_status.value = ''
            c, x, y = None, None, None
            monthly = True
            if color.value:
                try:
                    c = ecco_variable(color.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            if quiver_x.value:
                try:
                    x = ecco_variable(quiver_x.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            if quiver_y.value:
                try:
                    y = ecco_variable(quiver_y.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            output.clear_output()
            with output: plot(c, x, y)

    def on_clear_button(_):
        output.clear_output()
    
    load_button.on_click(on_load_button)
    clear_button.on_click(on_clear_button)
    display(hbox1, hbox2, hbox3, output)

print('Setup complete')

(run the next cell if you're running locally)

In [ ]:
%matplotlib ipympl
import math
import os
import requests
import datetime
import xgcm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from platform import system
from netrc import netrc
from urllib import request
from http.cookiejar import CookieJar
from io import StringIO
from warnings import filterwarnings
import ecco_v4_py as ecco
from ecco_v4_py import vector_calc, scalar_calc
from os.path import join,expanduser,exists,split

filterwarnings("ignore", category=FutureWarning)
downloads = os.path.join(os.path.expanduser('~'), 'Downloads/ECCO_V4r4_PODAAC')
_netrc = os.path.join(os.path.expanduser('~'), '_netrc' if system()=='Windows' else '.netrc')

# Information to look up a variable in EarthData by name
all_variables = ['global_mean_barystatic_sea_level_anomaly', 'global_mean_sterodynamic_sea_level_anomaly', 'global_mean_sea_level_anomaly', 'Pa_global', 'xoamc', 'yoamc', 'zoamc', 'xoamp', 'yoamp', 'zoamp', 'mass', 'xcom', 'ycom', 'zcom', 'sboarea', 'xoamc_si', 'yoamc_si', 'zoamc_si', 'mass_si', 'xoamp_fw', 'yoamp_fw', 'zoamp_fw', 'mass_fw', 'xcom_fw', 'ycom_fw', 'zcom_fw', 'mass_gc', 'xoamp_dsl', 'yoamp_dsl', 'zoamp_dsl', 'CS', 'SN', 'rA', 'dxG', 'dyG', 'Depth', 'rAz', 'dxC', 'dyC', 'rAw', 'rAs', 'drC', 'drF', 'PHrefC', 'PHrefF', 'hFacC', 'hFacW', 'hFacS', 'maskC', 'maskW', 'maskS', 'DIFFKR', 'KAPGM', 'KAPREDI', 'SSH', 'SSHIBC', 'SSHNOIBC', 'ETAN', 'EXFatemp', 'EXFaqh', 'EXFuwind', 'EXFvwind', 'EXFwspee', 'EXFpress', 'EXFtaux', 'EXFtauy', 'oceTAUX', 'oceTAUY', 'EXFhl', 'EXFhs', 'EXFlwdn', 'EXFswdn', 'EXFqnet', 'oceQnet', 'SIatmQnt', 'TFLUX', 'EXFswnet', 'EXFlwnet', 'oceQsw', 'SIaaflux', 'EXFpreci', 'EXFevap', 'EXFroff', 'SIsnPrcp', 'EXFempmr', 'oceFWflx', 'SIatmFW', 'SFLUX', 'SIacSubl', 'SIrsSubl', 'SIfwThru', 'SIarea', 'SIheff', 'SIhsnow', 'sIceLoad', 'SIuice', 'SIvice', 'ADVxHEFF', 'ADVyHEFF', 'DFxEHEFF', 'DFyEHEFF', 'ADVxSNOW', 'ADVySNOW', 'DFxESNOW', 'DFyESNOW', 'oceSPflx', 'oceSPDep', 'MXLDEPTH', 'OBP', 'OBPGMAP', 'PHIBOT', 'UVEL', 'VVEL', 'WVEL', 'THETA', 'SALT', 'RHOAnoma', 'DRHODR', 'PHIHYD', 'PHIHYDcR', 'UVELMASS', 'VVELMASS', 'WVELMASS', 'Um_dPHdx', 'Vm_dPHdy', 'ADVx_TH', 'ADVy_TH', 'ADVr_TH', 'DFxE_TH', 'DFyE_TH', 'DFrE_TH', 'DFrI_TH', 'ADVx_SLT', 'ADVy_SLT', 'ADVr_SLT', 'DFxE_SLT', 'DFyE_SLT', 'DFrE_SLT', 'DFrI_SLT', 'oceSPtnd', 'UVELSTAR', 'VVELSTAR', 'WVELSTAR', 'GM_PsiX', 'GM_PsiY']
all_datasets = ['GMSL_TIME_SERIES', 'GMAP_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'GEOMETRY_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'SSH_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'STRESS_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'MIXED_LAYER_DEPTH_LLC0090GRID', 'OBP_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'BOLUS_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID']
datasets = pd.Series(['GMSL_TIME_SERIES', 'GMSL_TIME_SERIES', 'GMSL_TIME_SERIES', 'GMAP_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'SBO_CORE_TIME_SERIES', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'GEOMETRY_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'OCEAN_3D_MIX_COEFFS_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'SSH_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'ATM_STATE_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'STRESS_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'HEAT_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'FRESH_FLUX_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_CONC_THICKNESS_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_VELOCITY_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_HORIZ_VOLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'SEA_ICE_SALT_PLUME_FLUX_LLC0090GRID', 'MIXED_LAYER_DEPTH_LLC0090GRID', 'OBP_LLC0090GRID', 'OBP_LLC0090GRID', 'OBP_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'OCEAN_VEL_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'TEMP_SALINITY_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'DENS_STRAT_PRESS_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_VOLUME_FLUX_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_MOMENTUM_TEND_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'OCEAN_3D_SALINITY_FLUX_LLC0090GRID', 'BOLUS_LLC0090GRID', 'BOLUS_LLC0090GRID', 'BOLUS_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID', 'OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID'],
                     index=all_variables)
timings = pd.Series(['Daily', 'Snapshot', 'Snapshot', 'None', 'None', 'All', 'Daily', 'Daily', 'Daily', 'Daily', 'All', 'All', 'Daily', 'Daily', 'Daily', 'All', 'Daily', 'All', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily', 'Daily'],
                    index=all_datasets)
granule_prefixes = pd.Series(['GLOBAL_MEAN_SEA_LEVEL', 'GLOBAL_MEAN_ATM_SURFACE_PRES', 'SBO_CORE_PRODUCTS', 'GRID_GEOMETRY', 'OCEAN_3D_MIXING_COEFFS', 'SEA_SURFACE_HEIGHT', 'ATM_SURFACE_TEMP_HUM_WIND_PRES', 'OCEAN_AND_ICE_SURFACE_STRESS', 'OCEAN_AND_ICE_SURFACE_HEAT_FLUX', 'OCEAN_AND_ICE_SURFACE_FW_FLUX', 'SEA_ICE_CONC_THICKNESS', 'SEA_ICE_VELOCITY', 'SEA_ICE_HORIZ_VOLUME_FLUX', 'SEA_ICE_SALT_PLUME_FLUX', 'OCEAN_MIXED_LAYER_DEPTH', 'OCEAN_BOTTOM_PRESSURE', 'OCEAN_VELOCITY', 'OCEAN_TEMPERATURE_SALINITY', 'OCEAN_DENS_STRAT_PRESS', 'OCEAN_3D_VOLUME_FLUX', 'OCEAN_3D_MOMENTUM_TEND', 'OCEAN_3D_TEMPERATURE_FLUX', 'OCEAN_3D_SALINITY_FLUX', 'OCEAN_BOLUS_VELOCITY', 'OCEAN_BOLUS_STREAMFUNCTION'],
                             index=all_datasets)

# Information to generate an xgcm grid
tile_connections = {'tile': {
    0: {'X': ((12, 'Y', False), (3, 'X', False)), 'Y': (None, (1, 'Y', False))},
    1: {'X': ((11, 'Y', False), (4, 'X', False)), 'Y': ((0, 'Y', False), (2, 'Y', False))},
    2: {'X': ((10, 'Y', False), (5, 'X', False)), 'Y': ((1, 'Y', False), (6, 'X', False))},
    3: {'X': ((0, 'X', False), (9, 'Y', False)), 'Y': (None, (4, 'Y', False))},
    4: {'X': ((1, 'X', False), (8, 'Y', False)), 'Y': ((3, 'Y', False), (5, 'Y', False))},
    5: {'X': ((2, 'X', False), (7, 'Y', False)), 'Y': ((4, 'Y', False), (6, 'Y', False))},
    6: {'X': ((2, 'Y', False), (7, 'X', False)), 'Y': ((5, 'Y', False), (10, 'X', False))},
    7: {'X': ((6, 'X', False), (8, 'X', False)), 'Y': ((5, 'X', False), (10, 'Y', False))},
    8: {'X': ((7, 'X', False), (9, 'X', False)), 'Y': ((4, 'X', False), (11, 'Y', False))},
    9: {'X': ((8, 'X', False), None), 'Y': ((3, 'X', False), (12, 'Y', False))},
    10: {'X': ((6, 'Y', False), (11, 'X', False)), 'Y': ((7, 'Y', False), (2, 'X', False))},
    11: {'X': ((10, 'X', False), (12, 'X', False)), 'Y': ((8, 'Y', False), (1, 'X', False))},
    12: {'X': ((11, 'X', False), None), 'Y': ((9, 'Y', False), (0, 'X', False))}
}}

# So that you don't have to remember whether to put dimension names in quotes or not
i, i_g, j, j_g, k, k_u, k_l, k_p1, tile, XC, YC, XG, YG, Z, Zp1, Zu, Zl, XC_bnds, YC_bnds, Z_bnds, tile, time, time_l = 'i', 'i_g', 'j', 'j_g', 'k', 'k_u', 'k_l', 'k_p1', 'tile', 'XC', 'YC', 'XG', 'YG', 'Z', 'Zp1', 'Zu', 'Zl', 'XC_bnds', 'YC_bnds', 'Z_bnds', 'tile', 'time', 'time_l'

# Used to select i, j, i_g, and j_g for quiver plots to space out data
skip = range(2, 88, 5)

# subplots[i] is the index of tile #i in the array of subplots
subplots = {
    'pacific': [(3, 0), (2, 0), (1, 0), (3, 1), (2, 1), (1, 1), (0, 2),
              (1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)],
    'atlantic': [(3, 2), (2, 2), (1, 2), (3, 3), (2, 3), (1, 3), (0, 2),
               (1, 0), (2, 0), (3, 0), (1, 1), (2, 1), (3, 1)],
}
# rotations[i] is the orientation of tile #i, as a multiple of 90 degrees
rotations = {
    'pacific': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'atlantic': [0, 0, 0, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1],
}

# Trigonometry for multiples of 90 degrees 
def cos90(angle):
    if angle % 4 == 0: return 1
    elif angle % 4 == 2: return -1
    else: return 0
def sin90(angle):
    if angle % 4 == 1: return 1
    elif angle % 4 == 3: return -1
    else: return 0

def adjust_timing(variable: str, timing: str) -> str:
    dataset = datasets[variable]
    if timing not in {'None', 'Monthly', 'Daily', 'Monthly Snapshot', 'Daily Snapshot'}:
        raise ValueError(str(timing) + ' is not a valid timing (select either Monthly, Daily, Monthly Snapshot, or Daily Snapshot)')
    elif timing in {'Monthly Snapshot', 'Daily Snapshot'} and timings[dataset] == 'Daily':
        raise ValueError('No snapshots available for ' + str(variable))
    elif timing in {'Monthly', 'Daily'} and timings[dataset] == 'Snapshot':
        raise ValueError('No monthly or daily averages available for ' + str(variable))
    elif timings[dataset] == 'None':
        return 'None'
    elif timing == 'None' and timings[dataset] == 'Snapshot':
        return 'Monthly Snapshot'
    elif timing == 'None' and timings[dataset] in {'Daily', 'All'}:
        return 'Monthly'
    else:
        return timing

def get_granule(granule: str, directory: str) -> str:
    file = os.path.join(directory, os.path.basename(granule))
    if not os.path.isfile(file):
        with requests.get(url=granule) as r:
            if r.status_code == 401:
                raise IOError('Incorrect EarthData login; check netrc')
            elif r.status_code // 100 != 2:
                raise IOError(r.text)
            with open(file, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024):
                    if chunk: f.write(chunk)
    return file

def ecco_dataset(dataset: str, start: datetime.date = None, end: datetime.date = None, timing: str = 'None'):
    short_timing_names = {'None': '', 'Monthly': '_MONTHLY', 'Daily': '_DAILY', 'Monthly Snapshot': '_SNAPSHOT', 'Daily Snapshot': '_SNAPSHOT'}
    long_timing_names = {'None': '', 'Monthly': '_mon_mean', 'Daily': '_day_mean', 'Monthly Snapshot': '_snap', 'Daily Snapshot': '_snap'}
    if timing not in short_timing_names:
        raise ValueError('Unrecognized timing: ' + str(timing))
    shortname = 'ECCO_L4_' + dataset + short_timing_names[timing] + '_V4R4'
    if 'LLC0090' in dataset:
        if timing == 'Monthly':
            start = datetime.date(start.year, start.month, 1)
            dates = [date.strftime('_%Y-%m') for date in pd.date_range(start, end, freq='MS')]
        elif timing == 'Daily':
            dates = [date.strftime('_%Y-%m-%d') for date in pd.date_range(start, end)]
        elif timing in {'Monthly Snapshot', 'Daily Snapshot'}:
            dates = [date.strftime('_%Y-%m-%dT000000') for date in pd.date_range(start, end)]
        elif timing == 'None':
            dates = ['']
        longnames = [granule_prefixes[dataset] + long_timing_names[timing] + date + '_ECCO_V4r4_native_llc0090.nc'
                    for date in dates]
    else:
        longnames = [granule_prefixes[dataset] + long_timing_names[timing] + '_ECCO_V4r4_1D.nc']
    granules = ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/' + shortname + '/' + longname
                for longname in longnames]
    granule_dir = downloads + '/' + shortname
    try: os.mkdir(granule_dir)
    except FileExistsError: pass
    files = [get_granule(granule, granule_dir) for granule in granules]
    array = xr.open_mfdataset(files, data_vars='minimal', coords='minimal', compat='override')
    if timing == 'Monthly':
        times = pd.DatetimeIndex(array.time)
        array = array.assign_coords(time=[str(t)[:7] for t in times])
    elif timing in {'Daily', 'Daily Snapshot', 'Monthly Snapshot'}:
        times = pd.DatetimeIndex(array.time)
        array = array.assign_coords(time=[str(t)[:10] for t in times])
    if timing == 'Monthly Snapshot':
        array = array.sel(time=[t for t in array.time.values if t[8:10] == '01'])
        array = array.assign_coords(time=[t[:7] for t in array.time.values])
    if timing in {'Daily Snapshot', 'Monthly Snapshot'}:
        array = array.rename(time=time_l)
    return array

def ecco_variable(variable: str, start: datetime.date = None, end: datetime.date = None, timing: str = 'None'):
    if variable not in all_variables:
        raise ValueError(str(variable) + ' is not an ECCO variable')
    timing = adjust_timing(variable, timing)
    if timing != 'None' and start is None and 'LLC0090' in datasets[variable]:
        raise ValueError('Enter a date to retrieve \'' + str(variable) + '\'')
    if type(start) == str:
        if len(start) == 7:
            start += '-01'
        start = datetime.datetime.strptime(start, '%Y-%m-%d')
    if type(end) == str:
        if len(end) == 7:
            end += '-01'
        end = datetime.datetime.strptime(end, '%Y-%m-%d')
    if end is None:
        end = start
    return ecco_dataset(datasets[variable], start, end, timing)[variable]

def print_value(array):
    if len(array.dims) > 1:
        dims = ', '.join(array.dims)
        raise ValueError('To get a single value, select or average along the remaining dimensions: ' + dims)
    else:
        value = array.values.item()
        if math.isnan(value):
            print('No value found (location is outside the bounds of the ocean)')
        else:
            if 'long_name' in array.attrs:
                print(array.long_name[:-1] + ': ' + str(value))
            else:
                print(value)
        for coord in {'XC', 'XG'}:
            if coord in array.coords:
                longitude = array[coord].values.item()
                print('Longitude: ' + str(abs(round(longitude, 3))) + ('°W' if longitude < 0 else '°E'))
                break
        for coord in {'YC', 'YG'}:
            if coord in array.coords:
                latitude = array[coord].values.item()
                print('Latitude: ' + str(abs(round(latitude, 3))) + ('°S' if latitude < 0 else '°N'))
                break
        for coord in {'Z', 'Zl', 'Zu', 'Zp1'}:
            if coord in array.coords:
                depth = array[coord].values.item()
                print('Depth: ' + str(round(-depth, 3)) + ' meters')
                break

def bounds(bottom, top): return range(bottom, top + 1)

geometry = ecco_dataset('GEOMETRY_LLC0090GRID')
xgcm_grid = xgcm.Grid(geometry, periodic=False, face_connections=tile_connections)

def interpolate(array, dim):
    if dim not in array.dims:
        raise ValueError(str(dim) + ' is not a dimension of the given variable')
    if len(array[dim]) < 2:
        raise ValueError('You need at least two coordinates to interpolate along a dimension')
    if dim == 'time':
        times = array[dim].values
        array = array.assign_coords({dim: range(len(times))})
        array = array.interp({dim: np.linspace(0.5, len(times) - 1.5, len(times) - 1)})
        return array.assign_coords({dim: times[1:]}).rename({dim: 'time_l'})
    if dim == 'time_l':
        times = array[dim].values
        array = array.assign_coords({dim: range(len(times))})
        array = array.interp({dim: np.linspace(0.5, len(times) - 1.5, len(times) - 1)})
        return array.assign_coords({dim: times[:-1]}).rename({dim: 'time'})
    grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(array.dims)
    if len(grid_dims) < 3 or any(len(array[dim]) < len(geometry[dim]) for dim in grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if dim in {'i', 'i_g', 'XC', 'XG'}:
        array_interp = xgcm_grid.interp(array.load(), 'X', keep_coords=True)
    elif dim in {'j', 'j_g', 'YC', 'YG'}:
        array_interp = xgcm_grid.interp(array.load(), 'Y', keep_coords=True)
    elif dim in {'k', 'k_u', 'k_l', 'k_p1', 'Z', 'Zp1', 'Zu', 'Zl'}:
        array_interp = xgcm_grid.interp(array.load(), 'Z', boundary='fill', fill_value=0, keep_coords=True)
    else: raise ValueError('Cannot interpolate along ' + str(dim))
    if 'time' in array.coords: array_interp = array_interp.assign_coords(time=array.time)
    elif 'time_l' in array.coords: array_interp = array_interp.assign_coords(time_l=array.time_l)
    return array_interp

def interpolate_2d(u, v):
    if {'i', 'j_g'} & set(u.dims):
        raise ValueError('The first input to interpolate_2d must be on the u-grid')
    if {'i_g', 'j'} & set(v.dims):
        raise ValueError('The second input to interpolate_2d must be on the v-grid')
    u_grid_dims, v_grid_dims = {'i_g', 'j', 'tile'}, {'i', 'j_g', 'tile'}
    if not (u_grid_dims <= set(u.dims)) or any(len(u[dim]) < len(geometry[dim]) for dim in u_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if not (v_grid_dims <= set(v.dims)) or any(len(v[dim]) < len(geometry[dim]) for dim in v_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    uv_interp = xgcm_grid.interp_2d_vector({'X': u.load(), 'Y': v.load()}, boundary='extend')
    u_interp, v_interp = uv_interp['X'], uv_interp['Y']
    if 'time' in u.coords: u_interp = u_interp.assign_coords(time=u.time)
    elif 'time_l' in u.coords: u_interp = u_interp.assign_coords(time_l=u.time_l)
    if 'time' in v.coords: v_interp = v_interp.assign_coords(time=v.time)
    elif 'time_l' in v.coords: v_interp = v_interp.assign_coords(time_l=v.time_l)
    return u_interp, v_interp

def difference(array, dim):
    if dim not in array.dims:
        raise ValueError(str(dim) + ' is not a dimension of the given variable')
    if len(array[dim]) < 2:
        raise ValueError('You need at least two coordinates to calculate difference along a dimension')
    if dim == 'time':
        return array.diff('time', label='upper').rename({'time': 'time_l'})
    if dim == 'time_l':
        return array.diff('time_l', label='lower').rename({'time_l': 'time'})
    grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(array.dims)
    if len(grid_dims) < 3 or any(len(array[dim]) < len(geometry[dim]) for dim in grid_dims):
        raise ValueError('You must calculate difference before selecting along grid dimensions')
    if dim in {'i', 'i_g', 'XC', 'XG'}:
        array_diff = xgcm_grid.diff(array.load(), 'X', keep_coords=True)
    elif dim in {'j', 'j_g', 'YC', 'YG'}:
        array_diff = xgcm_grid.diff(array.load(), 'Y', keep_coords=True)
    elif dim in {'k', 'k_u', 'k_l', 'k_p1', 'Z', 'Zp1', 'Zu', 'Zl'}:
        array_diff = -xgcm_grid.diff(array.load(), 'Z', boundary='fill', fill_value=0, keep_coords=True)
    else: raise ValueError('Cannot calculate difference along ' + str(dim))
    if 'time' in array.coords: array_diff = array_diff.assign_coords(time=array.time)
    elif 'time_l' in array.coords: array_diff = array_diff.assign_coords(time_l=array.time_l)
    return array_diff

def difference_2d(u, v):
    if {'i', 'j_g'} & set(u.dims):
        raise ValueError('The first input to interpolate_2d must be on the u-grid')
    if {'i_g', 'j'} & set(v.dims):
        raise ValueError('The second input to interpolate_2d must be on the v-grid')
    u_grid_dims, v_grid_dims = {'i_g', 'j', 'tile'}, {'i', 'j_g', 'tile'}
    if not (u_grid_dims <= set(u.dims)) or any(len(u[dim]) < len(geometry[dim]) for dim in u_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    if not (v_grid_dims <= set(v.dims)) or any(len(v[dim]) < len(geometry[dim]) for dim in v_grid_dims):
        raise ValueError('You must interpolate before selecting along grid dimensions')
    uv_diff = xgcm_grid.diff_2d_vector({'X': u.load(), 'Y': v.load()}, boundary='extend')
    u_diff, v_diff = uv_diff['X'], uv_diff['Y']
    if 'time' in u.coords: u_diff = u_diff.assign_coords(time=u.time)
    elif 'time_l' in u.coords: u_diff = u_diff.assign_coords(time_l=u.time_l)
    if 'time' in v.coords: v_diff = v_diff.assign_coords(time=v.time)
    elif 'time_l' in v.coords: v_diff = v_diff.assign_coords(time_l=v.time_l)
    return u_diff, v_diff

def colormap(data: xr.DataArray):
    cmin = np.nanpercentile(data, 10)
    cmax = np.nanpercentile(data, 90)
    if cmin < 0 and cmax > 0:
        cmax = np.nanpercentile(np.abs(data), 90)
        cmin = -cmax
        cmap = 'RdBu_r'
    else:
        cmap = 'viridis'

    return cmap, cmin, cmax

dimension_descriptions = {'i': 'Tile x-coordinate', 'j': 'Tile y-coordinate', 'k': 'Tile z-coordinate', 'Z': 'Depth (m)', 'tile': 'Plot area', 'time': 'Date'}
land_mask = mpl.colors.LinearSegmentedColormap.from_list('land_mask', ['#e0f0a0', '#ffffff'])

def update_plot(fig, data, x, y, selection, ocean_focus=None):
    names = data.data_vars.keys()
    title = widgets.Text(description='Plot title:')
    adjust_widgets = [title]
    ckind = data.c.dtype.kind
    if 'long_name' in data.c.attrs and 'vertical open fraction' in data.c.attrs['long_name']:
        ckind = 'b'
    cmap = widgets.Dropdown(description='Color map:', options=[
        ('viridis', 'viridis'), ('inferno', 'inferno'), ('cividis', 'cividis'), ('gray', 'binary'), ('gray (inverted)', 'gray'),
        ('pale', 'pink'), ('heat', 'gist_heat'), ('red-blue', 'RdBu_r'), ('seismic', 'seismic'), ('spectral', 'Spectral'),
        ('land mask', land_mask)
    ])
    if ckind == 'f':
        clabel = widgets.Text(description='Color units:')
        adjust_widgets.append(clabel)
    if {'u', 'v'} <= data.data_vars.keys():
        uvlabel = widgets.Text(description='Arrow units:')
        adjust_widgets.append(uvlabel)
    if ckind == 'f':
        adjust_widgets.append(cmap)
        if {'u', 'v'} <= data.data_vars.keys():
            acolor = widgets.Dropdown(description='Arrow color:', options=[('Black', 'k'), ('White', 'w')], value='k')
            adjust_widgets.append(acolor)
    display(widgets.HBox(adjust_widgets))

    fig.clf()
    # Select time/depth if possible before interpolating
    for dim in {'time', 'k'}:
        if dim in selection and dim in data.dims:
            data = data.sel({dim: selection[dim]})
    variables = dict(data.astype(float).data_vars)
    for (name, var) in variables.items():
        for dim in {'i_g', 'j_g', 'k_u', 'k_l', 'k_p1', 'time_l'}:
            if dim in var.dims:
                variables[name] = interpolate(var, dim)
    data = xr.Dataset(variables)
    # Second pass selection after interpolation changes dimensions
    for (dim, val) in selection.items():
        if dim in data.dims:
            data = data.sel({dim: val})
    if 'Z' in (x, y): data['Z'] = -data['Z']
    if ckind == 'f':
        cmap.value, cmin, cmax = colormap(data['c'])
    elif ckind == 'b':
        cmap.value, cmin, cmax = land_mask, 0, 1
    if {'u', 'v'} <= set(data.data_vars):
        x_skip, y_skip = math.ceil(len(data[x]) / 20), math.ceil(len(data[y]) / 20)
        quiver_x, quiver_y = data[x][(x_skip//2)::x_skip], data[y][(y_skip//2)::y_skip]
        uvmax = max(np.nanpercentile(np.abs(data.u), 90), np.nanpercentile(np.abs(data.v), 90))
    if 'tile' in data.dims:
        axes = fig.subplots(4, 4)
        if ckind == 'f':
            fig.set_size_inches(12.5, 10.1)
        elif ckind == 'b':
            fig.set_size_inches(10, 10.1)
        fig.subplots_adjust(wspace=0, hspace=0)
        for ax in axes.ravel():
            ax.axis('off')
        axes = [axes[row][col] for (row, col) in subplots[ocean_focus]]
        title.observe(lambda change: fig.suptitle(change['new'], x=0.435, y=0.92), names='value')
        meshes, quivers = [], []
        for tile, ax in enumerate(axes):
            if tile not in data.tile: continue
            ax.axis('on')
            ax.set_aspect('equal')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            c_rotated = np.rot90(data.c.sel(tile=tile).load(), rotations[ocean_focus][tile])
            meshes.append(ax.pcolormesh(data[x], data[y], c_rotated, cmap=cmap.value, vmin=cmin, vmax=cmax))
            if {'u', 'v'} <= set(data.data_vars):
                # Rotate head of each vector around the tile to the correct orientation
                u_rotated = np.rot90(data.u.sel({'tile': tile, x: quiver_x, y: quiver_y}), rotations[ocean_focus][tile])
                v_rotated = np.rot90(data.v.sel({'tile': tile, x: quiver_x, y: quiver_y}), rotations[ocean_focus][tile])
                # Rotate tail of each vector around the head by the same amount
                u_adjusted = u_rotated * cos90(rotations[ocean_focus][tile]) + v_rotated * sin90(rotations[ocean_focus][tile])
                v_adjusted = v_rotated * cos90(rotations[ocean_focus][tile]) - u_rotated * sin90(rotations[ocean_focus][tile])
                quivers.append(ax.quiver(quiver_x, quiver_y, u_adjusted, v_adjusted, scale=20*uvmax, width=0.006, clip_on=False))
        if ckind == 'f':
            cbar = fig.colorbar(meshes[0], ax=axes)
            clabel.observe(lambda change: cbar.set_label(change['new']), names='value')
            cmap.observe(lambda change: [mesh.set_cmap(change['new']) for mesh in meshes], names='value')
            if {'u', 'v'} <= set(data.data_vars):
                [quiver.set_color(acolor.value) for quiver in quivers]
                acolor.observe(lambda change: [quiver.set_color(change['new']) for quiver in quivers], names='value')
        if {'u', 'v'} <= set(data.data_vars):
            quiverkey = axes[6].quiverkey(quivers[6], 1.5, 0.5, 5*uvmax, f'{5*uvmax:.5g}')
            def set_quiverkey_label(change):
                nonlocal quiverkey
                quiverkey.remove()
                label = f'{5*uvmax:.5g}'
                if len(change['new']) > 0:
                    label += ' ' + change['new']
                quiverkey = axes[6].quiverkey(quivers[6], 1.5, 0.5, 5*uvmax, label)
            uvlabel.observe(set_quiverkey_label, names='value')
    else:
        ax = fig.subplots()
        if ckind == 'f':
            fig.set_size_inches(6.5, 5)
        elif ckind == 'b':
            fig.set_size_inches(5, 5)
        ax.set_xlabel(dimension_descriptions[x])
        ax.set_ylabel(dimension_descriptions[y])
        title.observe(lambda change: ax.set_title(change['new']), names='value')
        transpose = (x != data.c.dims[1] and y != data.c.dims[0])
        if (y in {'k', 'Z'}) or (transpose and y == 'i'):
            ax.yaxis.set_inverted(True)
            if 'v' in data.data_vars:
                data['v'] = -data['v']
        mesh_c = data.c.values
        if transpose: mesh_c = mesh_c.T
        mesh = ax.pcolormesh(data[x], data[y], mesh_c, cmap=cmap.value, vmin=cmin, vmax=cmax)
        if ckind == 'f':
            cbar = fig.colorbar(mesh)
            clabel.observe(lambda change: cbar.set_label(change['new']), names='value')
            cmap.observe(lambda change: mesh.set_cmap(change['new']), names='value')
        if {'u', 'v'} <= names:
            quiver_u = data.u.where(data[x].isin(quiver_x), drop=True).where(data[y].isin(quiver_y), drop=True)
            quiver_v = data.v.where(data[x].isin(quiver_x), drop=True).where(data[y].isin(quiver_y), drop=True)
            quiver_u, quiver_v = quiver_u.values, quiver_v.values
            if transpose: quiver_u, quiver_v = quiver_u.T, quiver_v.T
            quiver = ax.quiver(quiver_x, quiver_y, quiver_u, quiver_v, scale=20*uvmax, width=0.006)
            quiverkey = ax.quiverkey(quiver, 0.95, 1.05, 2*uvmax, f'{2*uvmax:.5g} ')
            def set_quiverkey_label(change):
                nonlocal quiverkey
                quiverkey.remove()
                label = f'{2*uvmax:.5g}'
                if len(change['new']) > 0:
                    label += ' ' + change['new']
                quiverkey = ax.quiverkey(quiver, 0.95, 1.05, 2*uvmax, label)
            uvlabel.observe(set_quiverkey_label, names='value')
            if ckind == 'f':
                quiver.set_color(acolor.value)
                acolor.observe(lambda change: quiver.set_color(change['new']), names='value')
        if x in {'time', 'time_l'}:
            ax.set_xticks(ax.get_xticks()[::3])

def make_coords_widget(selection, coords):
    output = widgets.Output()
    def show_coords(change):
        if change['new'] == 'Choose a value:':
            with output: display(coords)
        else:
            output.clear_output()
    selection.observe(show_coords, names='value')
    return output, show_coords

def plot(c: xr.DataArray = None, u: xr.DataArray = None, v: xr.DataArray = None):
    # If there is no color plot, plot land vs. ocean instead
    if c is None:
        c = ecco_variable('hFacC')
        if (u is None or 'k' not in u.dims) and (v is None or 'k' not in v.dims):
            c = c.sel(k=0)
    # If one of the arrow components isn't used, make it zero
    if u is not None and v is None:
        v = xr.DataArray(0, coords=u.coords, dims=u.dims)
    if v is not None and u is None:
        u = xr.DataArray(0, coords=v.coords, dims=v.dims)
        print(u)
    # plt.close() # Close other open plots to avoid having too many plots open at once
    # Merge variables into one dataset in order to perform uniform selection
    data = xr.Dataset({x_name: x for (x_name, x) in {'c': c, 'u': u, 'v': v}.items() if x is not None})
    if len(set(data.dims) - {'tile'}) < 2:
        raise ValueError('Must have at least two dimensions to make a plot')
    if any(len(data[dim]) == 0 for dim in data.dims):
        raise ValueError('Dimension with zero length')
    if {'i_g', 'j_g', 'k_l', 'k_u', 'k_p1', 'time_l'} & set(data.dims):
        grid_dims = {'i', 'i_g', 'j', 'j_g', 'tile'} & set(data.dims)
        if len(grid_dims) < 3 or any(len(data[dim]) < len(geometry[dim]) for dim in grid_dims):
            raise ValueError('In order for plotting to work correctly, you have to interpolate to grid cell centers before selecting along grid dimensions')
    selection_widgets = dict()
    selection_hboxes = []
    if 'tile' in data.dims:
        tile_options = [('Tile ' + str(tile), tile) for tile in data.tile.values]
        # Multi-tile plots only make sense if the data variables have both x- and y-coordinates
        if {'i', 'i_g'} & set(data.dims) and {'j', 'j_g'} & set(data.dims):
            tile_options = [('All tiles (Atlantic)', -1), ('All tiles (Pacific)', -2)] + tile_options
        tile_selection = widgets.Dropdown(description = 'Plot area:', options = tile_options)
        all_tiles_widgets = dict()
    if {'i', 'i_g'} & set(data.dims):
        i_selection = widgets.Dropdown(
            description = 'Tile x-coord:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Plot on x-axis',
        )
        i_coords = widgets.IntSlider(min=0, max=89)
        i_output, i_show_coords = make_coords_widget(i_selection, i_coords)
        selection_widgets['i'] = [i_selection, i_coords, i_output, i_show_coords]
        selection_hboxes.append(widgets.HBox([i_selection, i_output]))
    if {'j', 'j_g'} & set(data.dims):
        j_selection = widgets.Dropdown(
            description = 'Tile y-coord:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Plot on y-axis',
        )
        j_coords = widgets.IntSlider(min=0, max=89)
        j_output, j_show_coords = make_coords_widget(j_selection, j_coords)
        selection_widgets['j'] = [j_selection, j_coords, j_output, j_show_coords]
        selection_hboxes.append(widgets.HBox([j_selection, j_output]))
    if {'k', 'k_l', 'k_u', 'k_p1'} & set(data.dims):
        k_selection = widgets.Dropdown(
            description = 'Depth:',
            options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
            value = 'Choose a value:',
        )
        k_coords = widgets.SelectionSlider(options=[(str(int(-k)) + ' m', i) for (i, k) in enumerate(geometry.Z.values)])
        k_proportional = widgets.Checkbox(description='Proportional axis', value=False)
        k_output = widgets.Output()
        def k_show_coords(change):
            if change['new'] == 'Choose a value:':
                k_output.clear_output()
                with k_output: display(k_coords)
            elif change['old'] == 'Choose a value:':
                k_output.clear_output()
                with k_output: display(k_proportional)
        k_selection.observe(k_show_coords, names='value')
        selection_widgets['k'] = [k_selection, k_coords, k_output, k_show_coords]
        selection_hboxes.append(widgets.HBox([k_selection, k_output]))
        if 'tile' in data.dims:
            all_tiles_widgets['k'] = widgets.SelectionSlider(description='Depth:', options=k_coords.options)
    for dim in {'time', 'time_l'}:
        if dim in data.dims:
            t_selection = widgets.Dropdown(
                description = 'Date:',
                options = ['Plot on x-axis', 'Plot on y-axis', 'Choose a value:'],
                value = 'Choose a value:',
            )
            t_coords = widgets.SelectionSlider(options=data[dim].values)
            t_output, t_show_coords = make_coords_widget(t_selection, t_coords)
            selection_widgets['time'] = [t_selection, t_coords, t_output, t_show_coords]
            selection_hboxes.append(widgets.HBox([t_selection, t_output]))
            if 'tile' in data.dims:
                all_tiles_widgets['time'] = widgets.SelectionSlider(description='Date:', options=t_coords.options)
            break

    selection_output = widgets.Output()
    # 'change' means a change to the tile_selection widget's value (since tile_selection observes this function)
    def set_selection_widgets(change):
        selection_output.clear_output()
        if change['new'] < 0:
            with selection_output: display(*all_tiles_widgets.values())
        else:
            with selection_output: display(*selection_hboxes)
            for [selection, _, _, show_coords] in selection_widgets.values():
                # make coordinate sliders appear initially
                show_coords({'new': selection.value, 'old': 'Choose a value:'})
    set_selection_widgets({'new': tile_selection.value if 'tile' in data.dims else 0})
    if 'tile' in data.dims:
        tile_selection.observe(set_selection_widgets, names='value')

    plot_button = widgets.Button(description='Plot')
    clear_button = widgets.Button(description='Clear plot')
    plot_status = widgets.Label(value='')
    output = widgets.Output()
    fig = plt.figure()
    fig.set_size_inches(0.01, 0.01)

    def on_plot_button(_):
        plot_status.value = ''
        if 'tile' not in data.dims or tile_selection.value >= 0:
            selection = {dim: coords_widget.value
                         for (dim, [selection_widget, coords_widget, _, _]) in selection_widgets.items()
                         if selection_widget.value == 'Choose a value:'}
            if 'tile' in data.dims:
                selection['tile'] = tile_selection.value

            xaxis = [dim for (dim, [selection_widget, _, _, _]) in selection_widgets.items()
                     if selection_widget.value == 'Plot on x-axis']
            if len(xaxis) != 1:
                plot_status.value = 'One dimension must be selected to plot on the x-axis'
                return
            else: xaxis = xaxis[0]
            if xaxis == 'k' and k_proportional.value: xaxis = 'Z'

            yaxis = [dim for (dim, [selection_widget, _, _, _]) in selection_widgets.items()
                     if selection_widget.value == 'Plot on y-axis']
            if len(yaxis) != 1:
                plot_status.value = 'One dimension must be selected to plot on the y-axis'
                return
            else: yaxis = yaxis[0]
            if yaxis == 'k' and k_proportional.value: yaxis = 'Z'
        else:
            selection = {dim: widget.value for (dim, widget) in all_tiles_widgets.items()}
            xaxis, yaxis = 'i', 'j'
        output.clear_output()
        with output:
            if 'tile' not in data.dims or tile_selection.value >= 0: update_plot(fig, data, xaxis, yaxis, selection, None)
            elif tile_selection.value == -1: update_plot(fig, data, xaxis, yaxis, selection, 'atlantic')
            elif tile_selection.value == -2: update_plot(fig, data, xaxis, yaxis, selection, 'pacific')

    def on_clear_button(_):
        output.clear_output()
        fig.clf()
        fig.set_size_inches(0.01, 0.01)

    plot_button.on_click(on_plot_button)
    clear_button.on_click(on_clear_button)
    if 'tile' in data.dims:
        display(tile_selection)
    display(selection_output, widgets.HBox([plot_button, clear_button, plot_status]), output)
    plt.show()

def plot_utility():
    # plt.close()
    color = widgets.Text(description='Color plot:', value='THETA')
    quiver_x = widgets.Text(description='Arrow plot x:', value='UVELMASS')
    quiver_y = widgets.Text(description='Arrow plot y:', value='VVELMASS')
    hbox1 = widgets.HBox([color, quiver_x, quiver_y])
    start = widgets.DatePicker(description='Start date:', value=datetime.date(2017, 1, 1))
    end = widgets.DatePicker(description='End date:', value=datetime.date(2017, 1, 10))
    timing = widgets.Dropdown(options=['Monthly', 'Daily', 'Snapshot'], value='Daily', description='Timing:')
    hbox2 = widgets.HBox([start, end, timing])
    load_button = widgets.Button(description='Load data')
    clear_button = widgets.Button(description='Clear data')
    load_status = widgets.Label(value='')
    hbox3 = widgets.HBox([load_button, clear_button, load_status])
    output = widgets.Output()
    
    def on_load_button(_):
        if not (color.value or quiver_x.value or quiver_y.value):
            load_status.value = 'Enter variable names above'
        elif not (start.value and end.value):
            load_status.value = 'Enter start and end dates'
        elif start.value > end.value:
            load_status.value = 'Start date must be before end date'
        elif start.value < np.datetime64('1992-01-01'):
            load_status.value = 'Start date must not be before 1992'
        elif end.value >= np.datetime64('2018-01-01'):
            load_status.value = 'End date must not be after 2017'
        else:
            load_status.value = ''
            c, x, y = None, None, None
            monthly = True
            if color.value:
                try:
                    c = ecco_variable(color.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            if quiver_x.value:
                try:
                    x = ecco_variable(quiver_x.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            if quiver_y.value:
                try:
                    y = ecco_variable(quiver_y.value, start.value, end.value, timing.value)
                except ValueError as e:
                    load_status.value = str(e)
                    return
            output.clear_output()
            with output: plot(c, x, y)

    def on_clear_button(_):
        output.clear_output()
    
    load_button.on_click(on_load_button)
    clear_button.on_click(on_clear_button)
    display(hbox1, hbox2, hbox3, output)

# Login to EarthData from netrc file
try:
    username, _, password = netrc(file=_netrc).authenticators('urs.earthdata.nasa.gov')
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, 'urs.earthdata.nasa.gov', username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)
    with requests.get('https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc') as r:
        if r.status_code // 100 == 2:
            print('Login to EarthData successful')
        elif r.status_code == 401:
            print('Incorrect EarthData login; check netrc')
        else:
            print(r.text)
except FileNotFoundError:
    print('netrc file not found')

## Observing Array (Quant.)

First, let's load the datasets we were working with last time.

In [ ]:
# Load monthly temperature and salinity data with grid
folder = downloads+'/ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4'
files = os.listdir(folder) # list all files in the folder (each month of 2017)
paths = []
for file in files:
    paths.append(os.path.join(folder, file)) # make a list of all files with their complete path
ds_monthly = xr.open_mfdataset(paths)

ds_grid = xr.open_mfdataset(downloads+'/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_ds_TS = xr.merge((ds_grid,ds_monthly))

# Load monthly velocity data with grid
folder = downloads+'/ECCO_L4_OCEAN_3D_VOLUME_FLUX_LLC0090GRID_MONTHLY_V4R4'
files = os.listdir(folder) # list all files in the folder (each month of 2017)
paths = []
for file in files:
    paths.append(os.path.join(folder, file)) # make a list of all files with their complete path
ds_monthly = xr.open_mfdataset(paths)

ds_grid = xr.open_mfdataset(downloads+'/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_ds_vel = xr.merge((ds_grid,ds_monthly))

# Load monthly velocity data with grid
folder = downloads+'/ECCO_L4_OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID_MONTHLY_V4R4'
files = os.listdir(folder) # list all files in the folder (each month of 2017)
paths = []
for file in files:
    paths.append(os.path.join(folder, file)) # make a list of all files with their complete path
ds_monthly = xr.open_mfdataset(paths)

ds_grid = xr.open_mfdataset(downloads+'/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_ds_heat = xr.merge((ds_grid,ds_monthly))

Look back at the plots you made of meridional overturning circulation last time.

**Task:** In the following cell, answer: Around which latitude is the AMOC strongest? About how much does the strength of the AMOC vary over the year (in 2017)?

Let's investigate the spatial variation of the AMOC more closely. The following diagram shows the general path of the AMOC through the Atlantic ocean. The red line represents the warmer, northward-flowing water near the surface; the blue line represents the colder, southward-flowing water in the deep ocean.

<figure>
    <img src="Lab2Images/fig3.png" width="500"/>
    <figcaption>
        <a href="https://www.researchgate.net/figure/1-Schematic-representation-of-the-main-components-of-the-Atlantic-Meridional_fig1_341909781">Schematic representation of the AMOC</a>
    </figcaption>
</figure>

Using this diagram, and using your plots from Part 2, your goal will be to find specific grid cells in ECCO that exemplify the AMOC's motion. To that end, we have provided a plotting utility so that you can change the plotting parameters more quickly. The following code cells plot temperature flux using arrows, averaged over 2014-2017 and over depth.

In [ ]:
# Use for tiles 1 and 2
plot(None, ecco_ds_vel['UVELMASS'].mean('time'), ecco_ds_vel['VVELMASS'].mean('time'))

In [ ]:
# Use for tiles 10 and 11 (swap which coordinates are plotted on the axes)
plot(None, ecco_ds_vel['VVELMASS'].mean('time'), ecco_ds_vel['UVELMASS'].mean('time'))

The following code cells plot northward flux using color:

In [ ]:
# Use for tiles 1 and 2
plot(ecco_ds_vel['VVELMASS'].mean('time'))

In [ ]:
# Use for tiles 10 and 11 (swap which coordinates are plotted on the axes)
plot(-ecco_ds_vel['UVELMASS'].mean('time'))

(Note: if you get a warning message that says `RuntimeWarning: More than 20 figures have been opened`, run the following code cell to make it go away.)

In [ ]:
plt.close()

**Task:** Find around 5-10 points in the Atlantic ocean that typically have strong circulation associated with the AMOC. Try to choose points spread out throughout the Atlantic. For each point, find and save a plot that shows its circulation. In the list of `observations` below, include a value at this point which is positively associated with AMOC strength.

In [ ]:
depth = -np.round(ecco_ds_vel['Z']).astype(int)

observations = [ # Include 5-10 observation points separated by commas
    # Example: deep southward transport (positive UVELMASS) near the Caribbean is positively associated with the AMOC
    ecco_ds_vel['UVELMASS'].sel(tile=10, i_g=75, j=57).where(depth == 2491, drop=True),
    # TO-DO: Change the numbers (depth given in meters) and the direction of transport to select 5-10 different observations
]

For each of these observation points, we could imagine placing an real scientific instrument there to measure the strength of the AMOC. Using ECCO data, we can estimate what this *observing array* would measure for the AMOC's strength over time.

In [ ]:
observing_array = xr.concat(observations, 'n').squeeze('k')
plt.figure()
observing_array.sum('n').plot()
plt.title('Observing array')
plt.xlabel('Date')
plt.ylabel('Predicted AMOC strength')
plt.show()

**Task:** Compare the shape of the above plot of AMOC strength to the timeseries plot you made in Part 2. How well do they match up? If you find any large discrepancies for a particular month, try to explain them by comparing plots of AMOC strength and ocean velocity for that month.

### Heat transport

In [ ]:
# Use for tiles 1 and 2
trsp_x = (ecco_ds_heat['ADVx_TH'] + ecco_ds_heat['DFxE_TH']).mean('time').mean('k').compute()
trsp_y = (ecco_ds_heat['ADVy_TH'] + ecco_ds_heat['DFyE_TH']).mean('time').mean('k').compute()
plot(None, trsp_x, trsp_y)

In [ ]:
# Use for tiles 10 and 11 (swap which coordinates are plotted on the axes)
plot(None, trsp_y, trsp_x)

In [ ]:
# Use for tiles 1 and 2
plot(trsp_y)

In [ ]:
# Use for tiles 10 and 11 (swap which coordinates are plotted on the axes)
plot(-trsp_x)

**Task:** Find around 5-10 moorings in the Atlantic ocean that demonstrate the AMOC's northward heat transport. Modify the code from the last section to make a plot of predicted heat transport using your moorings. Compare with the timeseries of heat transport you produced in Part 2.

## Time Variation in the AMOC (Quant.)